In [5]:
import re
from collections import defaultdict

# Initialize dictionaries to store cumulative error and counts for each prediction type per subject
error_sum = defaultdict(lambda: defaultdict(float))
count = defaultdict(lambda: defaultdict(int))

# Parse the file
with open('./logs/debug_validate_log_split_1.txt', 'r') as file:
    for line in file:
        print(line)
       # Extract subject ID (supports alphanumeric format like 'P22' and 'ms1')
        subject_match = re.search(r"\['([a-zA-Z0-9]+)'\]", line)
        if not subject_match:
            print(f"Skipping line due to unmatched subject format: {line.strip()}")
            continue
        subject = subject_match.group(1)
        
        # Extract ground truth CPR rate
        gt_cpr_rate_match = re.search(r"GT_CPR_rate:\[(\d+\.?\d*)\]", line)
        if not gt_cpr_rate_match:
            print(f"Skipping line due to unmatched GT CPR rate format: {line.strip()}")
            continue
        gt_cpr_rate = float(gt_cpr_rate_match.group(1))

        # Extract different CPR rate errors
        cpr_rate_error_match = re.search(r"CPR_rate_error:(\d+\.?\d*)cpr/min", line)
        anonymous_error_match = re.search(r"anonymous_Pred_CPR_rate:\[(\d+\.?\d*)\]", line)
        low_pass_anonymous_error_match = re.search(r"Low_Pass_anonymous_Pred_CPR_rate:\[(\d+\.?\d*)\]", line)

        # If matches found, calculate and accumulate errors
        if cpr_rate_error_match:
            cpr_rate_error = float(cpr_rate_error_match.group(1))
            error_sum[subject]['Pred_CPR_rate'] += cpr_rate_error
            count[subject]['Pred_CPR_rate'] += 1

        if anonymous_error_match:
            anonymous_cpr_rate = float(anonymous_error_match.group(1))
            anonymous_error = abs(anonymous_cpr_rate - gt_cpr_rate)
            error_sum[subject]['anonymous_Pred_CPR_rate'] += anonymous_error
            count[subject]['anonymous_Pred_CPR_rate'] += 1

        if low_pass_anonymous_error_match:
            low_pass_anonymous_cpr_rate = float(low_pass_anonymous_error_match.group(1))
            low_pass_anonymous_error = abs(low_pass_anonymous_cpr_rate - gt_cpr_rate)
            error_sum[subject]['Low_Pass_anonymous_Pred_CPR_rate'] += low_pass_anonymous_error
            count[subject]['Low_Pass_anonymous_Pred_CPR_rate'] += 1

# Calculate the average CPR rate error for each subject and prediction type
average_errors = {subject: {} for subject in error_sum}
for subject in error_sum:
    for pred_type in error_sum[subject]:
        if count[subject][pred_type] > 0:
            average_errors[subject][pred_type] = error_sum[subject][pred_type] / count[subject][pred_type]

# Display the results
for subject, errors in average_errors.items():
    print(f"Subject: {subject}")
    for pred_type, avg_error in errors.items():
        print(f"  {pred_type} Average CPR Error: {avg_error:.2f} cpr/min")

['P14'],['s3'],GT_Depth:[1.7129065990447998],Pred_Depth:27.929636001586914,Depth_error:1.91mm,GT_CPR_rate:[4.5],Pred_CPR_rate:[4.0],CPR_rate_error:6.00cpr/min,anonymous_Pred_CPR_rate:[14.5],Low_Pass_anonymous_Pred_CPR_rate:[10.0]

['P14'],['s3'],GT_Depth:[1.7592449188232422],Pred_Depth:27.980457305908203,Depth_error:23.24mm,GT_CPR_rate:[6.0],Pred_CPR_rate:[3.5],CPR_rate_error:30.00cpr/min,anonymous_Pred_CPR_rate:[15.0],Low_Pass_anonymous_Pred_CPR_rate:[10.0]

['P14'],['s3'],GT_Depth:[4.4328484535217285],Pred_Depth:30.038991928100586,Depth_error:18.51mm,GT_CPR_rate:[6.0],Pred_CPR_rate:[3.0],CPR_rate_error:36.00cpr/min,anonymous_Pred_CPR_rate:[10.5],Low_Pass_anonymous_Pred_CPR_rate:[10.0]

['P14'],['s3'],GT_Depth:[7.764744758605957],Pred_Depth:33.540313720703125,Depth_error:19.52mm,GT_CPR_rate:[5.0],Pred_CPR_rate:[3.5],CPR_rate_error:18.00cpr/min,anonymous_Pred_CPR_rate:[12.0],Low_Pass_anonymous_Pred_CPR_rate:[9.5]

['P14'],['s3'],GT_Depth:[7.187206745147705],Pred_Depth:28.03302764892578

# Final average for all subjects